# Анализ лояльности пользователей Яндекс Афиши

- Автор: Литвинов Олег Игоревич
- Дата: 

## Цели и задачи проекта



## Этапы выполнения проекта

#### Загрузка данных и знакомство с ними
1. SQL запрос для выгрузки в датафрейм
2. Общая информация о данных
#### Предобработка данных
1. Преобразование к необходимому типу
2. Анализ пропусков
3. Обработка пропущенных значений
4. Проверка на явные и неявные дубликаты
#### Создание профиля пользователя
1. Агрегированные признаки, описывающие поведение и профиль каждого пользователя
2. Анализ данных в разрезе каждого пользователя
#### Исследовательский анализ данных
1. Исследование признаков первого заказа и их связи с возвращением на платформу
2. Исследование вернувшихся пользователей
4. Исследование поведения пользователей через показатели выручки и состава заказа
5. Исследование временных характеристик первого заказа и их влияния на повторные покупки
6. Корреляционный анализ количества покупок и признаков пользователя
#### Общие выводы и рекомендации
#### Финализация проекта и публикация в Git

In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine

In [2]:
# загружаем переменные из .env
load_dotenv()

db_config = {
    'user': os.getenv('DB_USER'),
    'pwd': os.getenv('DB_PASSWORD'), 
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT'),
    'db': os.getenv('DB_NAME')
}

In [3]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'],
)

In [4]:
engine = create_engine(connection_string)

In [5]:
query = '''
WITH set_config_precode AS (
  SELECT set_config('synchronize_seqscans', 'off', true)
)
SELECT  p.user_id,
        p.device_type_canonical,
        p.order_id,
        p.created_dt_msk AS order_dt,
        p.created_ts_msk AS order_ts,
        p.currency_code,
        p.revenue,
        p.tickets_count,
        EXTRACT(DAY FROM (created_dt_msk - LAG(created_dt_msk) OVER(
            PARTITION BY user_id 
            ORDER BY created_dt_msk
          ))) AS days_since_prev,
        p.event_id,
        e.event_name_code AS event_name,
        e.event_type_main,
        p.service_name,
        r.region_name,
        c.city_name
FROM afisha.purchases AS p
JOIN afisha.events e USING(event_id)
LEFT JOIN afisha.city c USING(city_id)
LEFT JOIN afisha.regions r USING(region_id)
WHERE p.device_type_canonical IN ('mobile', 'desktop') and e.event_type_main != 'фильм'
ORDER BY p.user_id
;
'''

In [6]:
df = pd.read_sql_query(query, con=engine) 
df.head()

,user_id,device_type_canonical,order_id,order_dt,order_ts,currency_code,revenue,tickets_count,days_since_prev,event_id,event_name,event_type_main,service_name,region_name,city_name
0,0002849b70a3ce2,mobile,4359165,2024-08-20,2024-08-20 16:08:03,rub,1521.94,4,NaN,169230,f0f7b271-04eb-4af6-bcb8-8f05cf46d6ad,театр,Край билетов,Каменевский регион,Глиногорск
1,0005ca5e93f2cf4,mobile,7965605,2024-07-23,2024-07-23 18:36:24,rub,289.45,2,NaN,237325,40efeb04-81b7-4135-b41f-708ff00cc64c,выставки,Мой билет,Каменевский регион,Глиногорск
2,0005ca5e93f2cf4,mobile,7292370,2024-10-06,2024-10-06 13:56:02,rub,1258.57,4,75.0,578454,01f3fb7b-ed07-4f94-b1d3-9a2e1ee5a8ca,другое,За билетом!,Каменевский регион,Глиногорск
3,000898990054619,mobile,1139875,2024-07-13,2024-07-13 19:40:48,rub,8.49,2,NaN,387271,2f638715-8844-466c-b43f-378a627c419f,другое,Лови билет!,Североярская область,Озёрск
4,000898990054619,mobile,972400,2024-10-04,2024-10-04 22:33:15,rub,1390.41,3,83.0,509453,10d805d3-9809-4d8a-834e-225b7d03f95d,стендап,Билеты без проблем,Озернинский край,Родниковецк
